# Uso de diferentes funciones de base en problemas de regresión

Extraido y adaptado por Pedro González Rodelas del repositorio de GitHub del módulo de Python [basis-expansions](https://github.com/madrury/basis-expansions) del autor [Matthew Drury](https://github.com/madrury).

Fecha de la primera versión: 30/03/2022

Sucesivas revisiones: 30/03/2022

Fecha de la última revisión: 30/3/2022

Para poder usar este módulo habrá que instalarlo previamente, ya sea directamente a través de GitHub, con la siguiente línea de comandos:

    pip install git+https://github.com/madrury/basis-expansions.git
    
o bien tras haber descargado los ficheros fuente a nuestro ordenador y ejecutando el fichero   `setup.py` con la línea de comandos

    pip install .
    
tras habernos situado en el directorio adecuado en una ventana de Anaconda Prompt del entorno de Python con el que vayamos a trabajar:

    cd basis-expansions-master


In [1]:
%pwd

'C:\\Users\\UX363\\Downloads\\Prácticas sobre splines-20220406\\Splines de Ajuste\\basis-expansions-master\\basis-expansions-master\\examples'

Los principiantes en el aprendizaje automático a menudo descartan los métodos de regresión después de aprender algoritmos más exóticos como `Boosting`, `Random Forests` o `Support Vector Machines (SVM)`; ¿por qué molestarse con un método lineal cuando  potentes métodos no paramétricos son también fácilmente disponibles?

Es fácil señalar que el carácter *lineal* en la `regresión lineal` no pretende transmitir que las predicciones del modelo resultante son lineales en sentido estricto, ¡solo los parámetros estimados!  

El modelador ciertamente puede capturar no linealidades en su regresión, como ya se ha visto también con la regresión polinomial y de tipo spline, ¡solo se necesita transformar adecuadamente los predictores en bruto!

Una respuesta común es que es un trabajo propenso a errores y molesto explorar los datos a mano y de alguna manera adivinar las transformaciones correctas de los predictores: ya que otros métodos lo hacen automáticamente.

Por lo general, el único método verdaderamente flexible que los principiantes aprenden para capturar no linealidades en la regresión es la `regresión polinómica`; lo cual es una verdadera lástima, ya que se trata del método de peor rendimiento disponible.

El propósito de este notebook sería el de difundir el conocimiento de mejores opciones para capturar `linealidades no lineales` (es decir combinaciones lineales de no linealidades) en modelos de regresión. La idea sería la de abogar por ejemplo por una adopción más generalizada del uso de splines lineales y cúbicos  en este campo de la Estadística.

### Agradecimientos

La idea de este notebook está basada en la siguiente respuesta [hxd1011](https://stats.stackexchange.com/users/113777/hxd1011)s [a una pregunta relacionada con técnicas de agrupamiento vs. splines](https://stats.stackexchange.com/questions/230750/when-should-we-discretize-bin-continuous-independent-variables-features-and-when) en [Validación Cruzada](https://es.wikipedia.org/wiki/Validaci%C3%B3n_cruzada) en un famoso [foro sobre Estadística](https://stats.stackexchange.com).

### Software

El autor [Matthew Drury](https://github.com/madrury) aprovechó la oportunidad de escribir un pequeño [módulo de Python](https://github.com/madrury/basis-expansions) que resulta bastante útil para poder usar diferentes bases en el desarrollo de técnicas de regresión. Se ajusta a la interfaz general de transformación del módulo por excelencia para Machine Learning (Aprendizaje Automático) `sklearn`, por lo que se puede utilizar también de manera combinada con otros procesos de alto nivel de este módulo más general.

## Bases de Expansión en Modelos de Regresión

Para capturar no linealidades en modelos de regresión, habitualmente necesitamos transformar algunos o todos los predictores.  Para evitar tener que tratar cada predictor como un caso especial que necesita investigación particularizada, nos gustaría tener alguna forma de aplicar una cierta *familia* de transformaciones muy general a nuestros predictores, que sea lo suficientemente flexible como para adaptarse (cuando el modelo es el adecuado) a una amplia variedad de formas.

Esto toma la forma general de una *expansión en funciones de base*.  La base aquí se utiliza en el sentido algebraico lineal: es decir un conjunto linealmente independiente de objetos.  En este caso nuestros objetos son *funciones*:

$$ B = f_1, f_2, \ldots, f_k $$

y creamos nuevos conjuntos de características y formas de todo tipo aplicando cada función en nuestra base a la característica dada:

$$ f_1(x), \ f_2(x), \ \ldots, \ f_k(x) $$


### Expansión Polinomial 

El ejemplo más común, y a menudo el *único* enseñado en cursos introductorios de modelado estadístico, o libros de texto en general, es el de la **regresión polinómica**.

En esta regresión polinómica elegimos como base un conjunto de términos polinómicos de grado creciente:

$$ f_1(x) = x, \ f_2(x) = x^2, \ \ldots, \ f_r(x) = x^r $$

Esto nos permite ajustar dichas curvas *polinómicas* a las formas características que queremos estudiar:

![Polinomios de Varios Grados](img/polynomial-various-degrees.png)

Desafortunadamente, la regresión polinómica tiene un buen número de problemas.  La más frecuentemente observada es una varianza muy alta (sensibilidad a los datos), especialmente cerca de los límites de los datos:

![Regresiones Polinomiales](img/polynomial-regressions-one-plot.png)

Arriba tenemos un conjunto de datos fijos, y tenemos regresiones polinómicas ajustadas y trazadas de varios grados.  La característica más llamativa es lo mal que los polinomios de mayor grado se ajustan a los datos cerca de los bordes.  ¡La varianza explota!  Esto es especialmente problemático en situaciones de alta dimensión donde casi *todos* los datos están cerca de los límites, debido a la denominada maldición de la dimensionalidad.

Otra forma de ver esto es trazar los residuos para cada punto de datos $x$ sobre muchas muestras de la misma población, ya que variamos el grado del polinomio que ajustamos a los datos:

![Residuos asociados a la Regresión Polinomial](img/polynomial-residuals-various-degrees.png)

Aquí vemos el mismo patrón de otras ocasiones, la inestabilidad en nuestros intentos suele comenzar en los bordes de los conjuntos de datos que pretendemos ajustar, y luego se expande hacia adentro, sobre todo a medida que aumentamos el grado.

Otra forma final de observar este efecto es estimar el error promedio de las regresiones polinómicas que se ajustan repetidamente a la misma población o conjunto de datos, a medida que se aumenta el grado:

![Polynomail Regression Average Error by Degree](img/polynomial-train-test-weird-signal.png)

De esta manera podemos ver claramente como la `regresión polinómica` eventualmente se `sobreajusta` drásticamente, incluso a este simple conjunto de datos unidimensionales.

Hay otros problemas con la regresión polinómica; por ejemplo, es inherentemente no local, ya que cambiando el valor de $y$ en un punto del conjunto de entrenamiento puede afectar el ajuste del polinomio en puntos de datos muy lejanos, lo que resulta en un acoplamiento estrecho a través del espacio de nuestros datos (a menudo denominado *rigidez*).  Puedes tener una idea de esto jugando con la app interactiva [suavizadores de diagramas de dispersión](http://madrury.github.io/smoothers/) alojado en este sitio web.

Los métodos que presentaremos en el resto de este notebook servirán de alguna manera para aliviar estos problemas que padece la regresión polinómica, y servirán como soluciones superiores a estos mismos problemas subyacentes.

### Expansiones troceadas

Probablemente lo primero que se les ocurre a la mayoría de los modeladores al reflexionar sobre otras formas de capturar efectos no lineales en regresión es **trocear** la variable predictora:

![Regresiones troceadas con varios números de Corte](img/bins-various-n-cuts.png)

En la regresión agrupada (o troceada) simplemente cortamos el rango de la variable predictora en intervalos de igual tamaño (aunque podríamos usar una regla más sofisticada, como cortar en intervalos a percentiles de la distribución marginal del predictor).  La pertenencia a cualquier intervalo se utiliza para crear un conjunto de variables indicadoras, a las cuales se les aplica la correspondiente regresión.  En el caso de un único predictor, esto da como resultado que nuestra regresión prediga al menos el valor medio de $y$ en cada trozo o agrupamiento.

Este troceado tiene sus [problemas conceptuales] obvios (https://stats.stackexchange.com/questions/68834/what-is-the-benefit-of-breaking-up-a-continuous-predictor-variable).  

De manera más prominente, esperamos que la mayoría de los fenómenos que estudiamos varíen continuamente con las entradas.  La regresión troceada no crea funciones continuas del predictor, por lo que en la mayoría de los casos esperaríamos que hubiera algún sesgo inevitable dentro de cada trozo.  En el caso simple en que la verdadera relación sea monótona en un intervalo, esperamos estar subestimando el valor en el lado izquierdo de cada trozo, y en el lado derecho se espera que predigamos en exceso.

![Regresión troceada vs. Señal Real](img/binned-regression-vs-truth.png)

Aun así, la regresión troceada es bastante popular, ya que es relativamente fácil de ver, implementar y explicar, y a menudo hace un trabajo lo suficientemente bueno como para capturar el comportamiento no lineal de la relación de respuesta predictora.  Sin embargo, hay mucho mejores opciones. Veremos más adelante que otros métodos son más atractivos y hacen un mejor trabajo, al capturar el poder predictivo en los datos, incluso con parámetros menos estimados.

### Splines Lineales (funciones lineales a trozos y continuas)

Como primer paso hacia una expansión de tipo general de una base *continua* no paramétrica, nos gustaría ajustar una función lineal **a trozos** a nuestros datos.  Esto resulta ser bastante fácil de hacer usando ciertas traslaciones de una cierta función plantilla genérica, de tipo **potencia truncada lineal**: $f(x) = max(0, x)$

![Elementos de la base de splines lineales](img/pl-basis-functions.png)

Tomando combinaciones lineales de funciones de este tipo, podremos crear muchas formas lineales a trozos que se adapten a nuestros datos.

![Ejemplo de Combinación Lineal de funciones de base](img/pl-linear-combination.png)

Si elegimos un conjunto fijo de puntos donde permitimos que la pendiente del segmento lineal cambie, entonces podemos usarlos como funciones base en nuestra regresión:

$$ \begin{align}
   f_0(x) &= 1 \\
   f_1(x) &=  x\\
   f_2(x) &= max(0, x - k_1)\\
   &\vdots \\
   f_r(x) &= max(0, x - k_r) 
\end{align}$$

Como resultado podremos ajustar una función continua y lineal a trozos a nuestros datos:

![Aproximación lineal a trozos con n-nodos](img/pl-various-n-knots.png)

Los valores $\{k_1, k_2, \ldots, k_r\}$, que son las coordenadas de $x$ en las que los sucesivos segmentos podrán cambiar de pendiente, serán llamados **nodos**.

Nótese que incluimos también las funciones de base $f_0(x) = 1$ y $f_1(x)=x$ que permitirán que el spline pueda tomar ciertos valores con pendiente no nula justo *antes* del primer nodo. Si no hubiésemos incluido estas funciones de base, nos habríamos visto obligados a usar una pendiente cero en el intervalo $(\infty, k_1)$.

Los parámetros estimados para los elementos de estas funciones de base tienen una interpretación simple, representan el *cambio de pendiente* al pasar de la parte izquierda a la derecha de cada nodo.

La adaptación de trozos lineales a trozos en lugar de polinomios de grado alto evita la explosión de varianza al estimar un gran número de parámetross:

![Entrenamiento y Errores con splines lineales](img/pl-train-test-weird-signal.png)

Vemos que a medida que aumenta el número de nodos, la spline lineal comienza a *sobreajustarse*, pero mucho más lentamente que la regresión polinómica con el mismo número de parámetros.

Con este tipo de ajuste lineal a trozos la varianza no tiende a acumularse en ningún área concreta (como lo hizo en el caso polinómico, siempre acumulándose cerca de los límites de los datos). En cambio, ahora las zonas de alta varianza dependen de los datos específicos que estamos ajustando.

![Resíduos para diferente número de nodos](img/pl-residuals-various-degrees.png)

### Splines Cúbicos Naturales

Nuestro último ejemplo de expansión en funciones de base será la de los **splines cúbicos naturales**.  Las splines cúbicas están motivadas por la filosofía de que la mayoría de los fenómenos que nos gustaría estudiar varían como una función *suave* (suficientemene derivable) de sus entradas.  Si bien los splines lineales eran funciones continuas, sus derivadas sí que cambiaron abruptamente en los nodos, y nos gustaría alguna forma de ajustar las curvas a nuestros datos donde el cambio sea menos violento en dichos nodos.

Una **spline cúbica natural** es una curva que de partida es continua, pero también tiene primeras derivadas continuas (la recta tangente no hace cambios bruscos en ningún punto), así como segundas derivadas continuas (la tasa de rotación de las líneas tangentes no hace cambios abruptos); y es igual a un polinomio cúbico excepto en los puntos donde se permite altas discontinuidades pero de orden superior, estos puntos son los **nodos**. Además en el caso del spline cúbico natural, requerimos también que la curva sea *lineal* más allá del conjunto de nodos: es decir, a la izquierda del primer nodo y a la derecha del nodo final.

![Ejemplo de Spline Cúbico Natural](img/natural-cubic-spline.png)

Omitiremos escribir la forma exacta de las funciones base para spline cúbico natural, el lector interesado puede consultar [The Elements of Statistical Learning](http://web.stanford.edu/~hastie/ElemStatLearn/) o el [código fuente del módulo](https://github.com/madrury/basis-expansions/blob/master/basis_expansions.py#L205) usado en este notebook.

![Splines Cúbicos Naturales ajustando los datos](img/natural-cubic-splines-various-knots.png)

Todos pensaron en un primer momento que parecen más complejos, es importante darse cuenta de que las restricciones de continuidad en la forma de la spline son fuertes.  Si bien la estimación de una spline lineal con $r$  nodos utiliza $r+1$ parámetros (ignorando la intercepción), la estimación de una spline cúbica toma solo $r$ parámetros.  De hecho, en la imagen de arriba, la spline con un solo nodo es una *línea*; esto se debe a que las pendientes de la línea en la parte izquierda y la derecha también deben coincidir.

Las restricciones lineales cerca del borde de los datos están destinadas a evitar que la spline se sobreajuste cerca de los límites de los datos, como tiende a hacer la regresión polinómica:

![Residuos de las Splines Cúbicas Naturales](img/natural-cubic-splines-various-degrees.png)

## Experimentos

Describimos a continuación cuatro experimentos comparando el comportamiento de la base anterior de los splines. En cada experimento ajustamos un splines de regresión al conjunto de datos usando una de las expansiones anteriores. Los grados de libertad (número estimado de parámetros a calcular) se ha ido variando, y la bondad del ajuste se ha medido en un conjunto de datos seleccionado[^1]. Este procedimiento ha sido luego repetido muchas veces[^2] para cada tipo de expansión y número de grados de libertad, y la media y varianza del error en los datos seleccionados se ha calculado.

Cada uno de los cuatro experimentos pretendía ajustar a conjuntos de datos muestreados a partir de una señal verdadera diferente:

  - Una línea con ruido gausiano aleatorio añadido.
  - Una curva sinusoidal con ruido gausiano aleatorio añadido.
  - Una curva continua algo extraña curve de diseño mayormente aleatorio, más ruido gaussiano añadido.
  - Una curva sinusoidal discontinua (partida) con ruido gausiano aleatorio añadido.

### Ajuste de una Linea con ruido

Como primer experimento, ajustemos una verdadera linea. Este sería el experimento más simple posible para nuestros métodos de regresión.

![Señal Lineal con Ruido Gausiano](img/linear-signal.png)

Veamos cómo varía el error de entrenamiento y prueba a medida que aumentamos el número de parámetros de ajuste para cada una de nuestras expansiones de base.

![](img/linear-signal-binning.png)
![](img/linear-signal-polynomial.png)
![](img/linear-signal-pl.png)
![](img/linear-signal-splines.png)

La expansión polinómica eventualmente se sobreajusta **drásticamente**, mientras que las otras tres transformaciones solo se sobreajustan gradualmente.  Ninguno de los modelos tiene ningún sesgo, por lo que se produce el error de prueba mínimo esperado para el número mínimo de parámetros de ajuste.

Las transformaciones cúbicas de binning, lineales por piezas y por piezas funcionan casi igual de bien en términos de error de prueba promedio en todo momento, y parecen tener aproximadamente la misma variación en el error de prueba (es decir, las bandas alrededor del área de las curvas de error de prueba tendrán aproximadamente el mismo ancho).

Podemos obtener una comparación más clara trazando solo los errores de prueba en el mismo eje:

![](img/linear-signal-all.png)

Entre dos y quince parámetros de ajuste, todos los métodos, excepto la regresión polinómica, funcionan aproximadamente bien y sobreajustan los datos de entrenamiento a la misma velocidad.  Entonces, en este caso simple, si solo vamos por el rendimiento de la prueba, no importa tanto lo que elijamos.

Para examinar la varianza de los métodos, podemos trazar el ancho de las bandas de varianza todas en el mismo eje:

![](img/linear-signal-variance.png)

En este ejemplo, la varianza del error de prueba permanece igual para todas las expansiones de base en todo el rango de complejidad, con la obvia excepción de la regresión polinómica, donde la varianza explota como era de esperar.

### Ajuste de una curva Sinusoidal con Ruido

Nuestro próximo experimento ajustará las diversas expansiones de base a una señal sinusoidal:

![Señal Sinusoidal con Ruido Gausiano](img/sin-signal.png)

Los gráficos de error de entrenamiento y prueba se crean de la misma manera que para la señal lineal:

![](img/sin-signal-binning.png)
![](img/sin-signal-polynomial.png)
![](img/sin-signal-pl.png)
![](img/sin-signal-splines.png)

La diferencia más notable en este caso es que al empezar con un número bajo de parámetros estimados, todos los modelos están muy sesgados: mientras que los ajustes a la función lineal comenzaban en sus errores de prueba y entrenamiento más bajos, para esta señal sinusoidal cada regresión con expansiones de bases diferentes comienza con un error alto y disminuye rápidamente a medida que más parámetros ingresan en el modelo, y el sesgo disminuye.

Las tasas de sobreajuste cuentan la misma historia que antes.  La regresión polinómica se sobreajusta drásticamente, mientras que los otros se sobreajustan lentamente.

Trazar las curvas de error de retención en la misma trama resalta otra característica interesante de la historia:

![](img/sin-signal-all.png)

La regresión agrupada se ajusta a los datos mucho más lentamente que la polinómica y las de tipo spline. Mientras que los splines logran su error de prueba promedio más bajo en alrededor de cinco parámetros estimados, se necesitan alrededor de dieciocho subintervalos para minimizar el error de prueba promedio.  Además, el error de prueba mínimo para las regresiones polinómica y spline es *menor* que para los troceados (y de nuevo, este mínimo se logra con *menos* parámetros estimados).

![](img/sin-signal-variance.png)

Una historia similar se aplica a las variaciones de error de prueba: la transformación de troceado o agrupamiento tiene una varianza ligeramente mayor que los dos métodos de tipo spline.

### Ajuste de una curva extraña con ruido adicional

El siguiente ejemplo se trata de una cierta función rara construida a mano para que tenga una forma inusual

```python
def weird_signal(x):
    return (x*x*x*(x-1) 
            + 2*(1/(1 + np.exp(-0.5*(x - 0.5)))) 
            - 3.5*(x > 0.2)*(x < 0.5)*(x - 0.2)*(x - 0.5)
            - 0.95)
```

La gráfica de esta función posee tramos del dominio con diferentes características de tipo exponencial, cuadrático y lineal.

![Señal Extraña con Ruido Gausiano](img/weird-signal.png)

Seguimos el experimento habitual en este punto, aquí están las curvas de entrenamiento y prueba para los diversos métodos de expansión de base que estamos investigando.

![](img/weird-signal-binning.png)
![](img/weird-signal-polynomial.png)
![](img/weird-signal-pl.png)
![](img/weird-signal-splines.png)

La historia aquí se repite con las mismas cuestiones generales que se tenían en el ejemplo anterior:

  - La expansión de la base cuadrática eventualmente se sobreajusta drásticamente.
  - Las otras expansiones de base se sobreajustan lentamente.
  - Para un número muy pequeño de parámetros estimados, cada una de las expansiones de la base está sesgada.
  - La varianza en los errores de prueba es generalmente estable a medida que se agregan más parámetros al modelo, *excepto* para la regresión polinómica, que explota.
  
La comparación de todos los errores de prueba en la misma gráfica resalta nuevamente puntos similares a la curva sinusoidal

![](img/weird-signal-all.png)

El sesgo de la regresión troceada disminuye más lentamente que la de los métodos de tipo spline, y su error de retención mínimo es mayor que el de los splines y se logra con más parámetros estimados.  La spline lineal y cúbica generalmente logran un error de prueba similar para el mismo número de parámetros estimados, o subintervalos considerados.

![](img/weird-signal-variance.png)

Las varianzas para los modelos no polinómicos son estables, con un indicio de que la regresión agrupada tiene una mayor varianza general en un rango de complejidades del modelo.

### Ajustando una Curva Sinusoidal Discontinua

Para nuestro ejemplo final, probaremos ahora con una función discontinua.

```python
cutpoints = sorted(np.random.uniform(size=6))
def broken_sin_signal(x):
    return  (np.sin(2*np.pi*x) 
             - (cutpoints[0] <= x)*(x <= cutpoints[2]) 
             - (cutpoints[1] <= x)*(x <= cutpoints[2])
             - 2*(cutpoints[3] <= x)*(x <= cutpoints[4]))
```

Para crear esta función, hemos tomado la curva sinusoidal de antes, y hemos creado discontinuidades desplazando los intervalos del gráfico hacia arriba o hacia abajo.  El resultado es una curva sinusoidal discontinua

![Señal Sinusoidal Discontinua con Ruido Gausiano](img/broken-signal.png)

Repitamos ahora nuestros mismos experimentos anteriores para este caso discontinuo, y veamos si las discontinuidades cambian los patrones que hemos estado observando entre las diferentes expansiones de base.

![](img/broken-signal-binning.png)
![](img/broken-signal-polynomial.png)
![](img/broken-signal-pl.png)
![](img/broken-signal-splines.png)

La principal diferencia en este ejemplo es la dificultad para disminuir el sesgo del ajuste.  Dado que los modelos que estamos ajustando son todos continuos, es *imposible* que cualquiera de ellos sea imparcial.  

Lo mejor que podemos hacer es atrapar las discontinuidades entre nodos cada vez más cercanos (en el caso de los splines), o dentro de un pequeño subintervalo.  De hecho, incluso con 29 parámetros estimados, el error de prueba sigue disminuyendo (lentamente), lo que indica que nuestros modelos aún pueden hacer un mejor trabajo disminuyendo el sesgo de sus ajustes, antes de verse abrumados con una varianza desmesurada.

De lo contrario, no hay diferencias sorprendentes en los patrones que observamos al comparar las diversas expansiones.

![](img/broken-signal-all.png)

Comparando los errores de prueba de los diversos métodos, esta vez las expansiones agrupadas son notablemente peores que los otros métodos en toda la gama de complejidades. Por el contrario, los dos modelos de spline son en su mayoría equivalentes en rendimiento.

![](img/broken-signal-variance.png)

Finalmente, la varianza del modelo troceado también es significativamente peor que la de los otros métodos, mientras que las dos metodologías spline son equivalentes.

## Conclusiones

En este notebook hemos comparado cuatro aproximaciones diferentes para la captura del comportamiento general no lineal en regresión.

### Troceado o agrupamiento

Esta técnica de troceado, aunque conceptualmente simple, se ha visto que sufre de unos cuantos problemas en comparación con los otros métodos:

  - Con un pequeño número de parámetros estimados, la regresión agrupada sufrió un sesgo más alto que sus competidores.  Se observó en múltiples experimentos que para lograr un error de retención mínimo en un mayor número de parámetros estimados que los otros métodos, y además, a menudo el error mínimo logrado por el método de agrupamiento fue *mayor* que el mínimo logrado por los otros métodos.
  
  - El error de retención de la regresión agrupada a menudo tenía una varianza más alta que el del otro método.  Esto significa que, incluso si funcionó tan bien como otro método *en promedio*, cualquier regresión *individual* agrupada es menos confiable que si se usa otro método.
  
Además, el método de regresión agrupada tiene la desventaja de producir funciones *discontinuas*, mientras que esperamos que la mayoría de los procesos que encontramos en la naturaleza o las finanzas varíen continuamente en sus entradas.  Esto es filosóficamente recomendable, y también explica parte del sesgo observado al comparar las regresiones de agrupamiento con las otras expansiones de base.

### Regresión Polinomial 

El principal problema con la regresión polinómica es su inestabilidad una vez que se supera un número moderado de parámetros estimados.  Las regresiones polinómicas son *altamente* sensibles al ruido en nuestros datos, y en cada ejemplo se vieron eventuales y violentos sobreajustes; esta característica no se observó en ninguno de los otros métodos que probamos.

La regresión polinómica tampoco acumula varianza de manera uniforme a lo largo del soporte de los datos.  Vimos que los ajustes polinómicos se vuelven altamente inestables cerca de los límites de los datos disponibles.  Esto puede ser un problema extraño en situaciones de dimensiones superiores (aunque no exploramos esto aquí), donde la denominada *maldición de la dimensionalidad* nos dice que la *mayoría* de nuestros datos se encontrarán cerca del límite.

### Splines Lineales y Cúbicos 

Estos métodos funcionaron bien en toda la gama de nuestros experimentos:

  - Se sobreajustan lentamente, especialmente en comparación con la regresión polinómica.
  - Por lo general, logran un error de retención más bajo que las regresiones agrupadas, y logran su error de retención mínimo con un número menor de parámetros estimados.
  - Su error de retención es generalmente el más bajo de los métodos que probamos, lo que hace que cada trozo de regresión *individual* sea más confiable.
  
Los dos modelos de tipo spline son claros ganadores sobre las regresiones troceadas y polinómica.  Esto es una verdadera lástima, porque a menudo no se enseñan en cursos introductorios de modelado predictivo, a pesar de que (al menos la spline lineal a trozos) son tan fáciles de implementar como la regresión polinómica y la agrupada (o troceada).

Esperamos pues haberle convencido de utilizar estos métodos de tipo spline en su propio trabajo de modelado. De hecho no vemos ninguna razón real para ajustar modelos mediante expansiones polinómicas, o bien agrupadas en trozos, en ningún modelo predictivo o inferencial, que no pueda hacerse mejor con modoelos de tipo spline.